In [27]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import xmltodict
import json
import os

class CEMTaggerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("CEM Tagger")

        # Maximizar la ventana al inicio
        self.root.attributes('-zoomed', True)

        # Crear una etiqueta de bienvenida con el texto adicional
        welcome_title = "Bienvenida al CEM Tagger"
        welcome_text = ("\n"
                        "Este es un prototipo en desarrollo que permite hacer el etiquetado de referentes \n"
                        "y correferentes para un corpus previamente etiquetado morfosintácticamente. \n"
                        "Este proyecto es desarrollado en el Semillero Corpus Ex Machina de la Universidad de Antioquia.")
        self.label_title = tk.Label(root, text=welcome_title, font=("Arial", 24), justify=tk.CENTER)
        self.label_title.pack(pady=10)

        self.label_text = tk.Label(root, text=welcome_text, font=("Arial", 12), justify=tk.LEFT)
        self.label_text.pack(pady=10)

        # Crear los botones más grandes con estilos modificados
        self.convert_button = tk.Button(root, text="Convertir XML a JSON", command=self.convert_xml_to_json, borderwidth=1, relief="raised", width=35, height=3, font=("Arial", 16), bg="#4CAF50", fg="white")
        self.convert_button.place(relx=0.5, rely=0.61, anchor=tk.CENTER)

        self.tag_button = tk.Button(root, text="Etiquetar Referentes y Correferentes", command=self.tag_references, borderwidth=1, relief="raised", width=35, height=3, font=("Arial", 16), bg="#2196F3", fg="white")
        self.tag_button.place(relx=0.5, rely=0.4, anchor=tk.CENTER)

    def convert_xml_to_json(self):
        # Abrir el cuadro de diálogo para seleccionar un archivo XML
        file_path = filedialog.askopenfilename(filetypes=[("XML files", "*.xml")])
        if not file_path:
            return  # El usuario canceló la selección

        # Verificar si la extensión es correcta
        if not file_path.lower().endswith('.xml'):
            messagebox.showwarning("Formato Incorrecto", "El archivo seleccionado no es un archivo XML válido.")
            return

        # Intentar convertir XML a JSON
        try:
            with open(file_path, 'r', encoding='utf-8') as xml_file:
                xml_content = xml_file.read()
                json_data = xmltodict.parse(xml_content, encoding='utf-8', process_namespaces=True)
        except Exception as e:
            messagebox.showerror("Error de Conversión", f"No se pudo convertir el archivo XML a JSON: {str(e)}")
            return

        # Mostrar el JSON resultante (esto es solo un ejemplo, puedes hacer lo que desees con `json_data`)
        messagebox.showinfo("Conversión Exitosa", "El archivo XML se ha convertido correctamente a JSON.")

        # Pedir al usuario que ingrese el nombre del archivo para guardar
        filename = simpledialog.askstring("Guardar JSON", "Ingrese el nombre del archivo JSON:")
        if not filename:
            return  # El usuario canceló la entrada del nombre

        # Agregar la extensión .json si no está presente
        if not filename.endswith('.json'):
            filename += '.json'

        # Obtener la ruta de destino para guardar el archivo JSON
        save_path = filedialog.asksaveasfilename(defaultextension=".json", filetypes=[("JSON files", "*.json")], initialfile=filename)
        if not save_path:
            return  # El usuario canceló la selección de la ubicación de guardado

        # Guardar el archivo JSON
        try:
            with open(save_path, 'w', encoding='utf-8') as json_file:
                json.dump(json_data, json_file, indent=4)
            messagebox.showinfo("Guardado Exitoso", f"El archivo JSON '{os.path.basename(save_path)}' se ha guardado correctamente.")
        except Exception as e:
            messagebox.showerror("Error al Guardar", f"No se pudo guardar el archivo JSON: {str(e)}")

    def tag_references(self):
        print("Etiquetar Referentes y Correferentes presionado")
        # Aquí puedes agregar la lógica para etiquetar referentes y correferentes

if __name__ == "__main__":
    root = tk.Tk()
    app = CEMTaggerApp(root)
    root.mainloop()


la versión anterior no tiene errores para xml a json. Se procede con la funcionalidad de etiquetado

In [39]:
import tkinter as tk
from tkinter import messagebox, filedialog

class ReferenceTaggingOptions(tk.Frame):
    def __init__(self, parent, on_close_callback):
        super().__init__(parent)
        self.configure(bg="#f0f0f0")

        # Título de las opciones de etiquetado de referentes
        self.label_title = tk.Label(self, text="Opciones de Etiquetado de Referentes", font=("Arial", 20), bg="#f0f0f0")
        self.label_title.pack(pady=20)

        # Botón para etiquetar referentes
        self.btn_tag_references = tk.Button(self, text="Etiquetar Referentes", command=self.tag_references, bg="#2196F3", fg="white", font=("Arial", 16), width=35, height=3, relief="raised")
        self.btn_tag_references.pack(pady=10)

        # Botón para cancelar y cerrar la ventana
        self.btn_cancel = tk.Button(self, text="Cancelar", command=self.cancel, bg="#FF5733", fg="white", font=("Arial", 16), width=35, height=3, relief="raised")
        self.btn_cancel.pack(pady=10)

        # Callback para cerrar la ventana y volver a la principal
        self.on_close_callback = on_close_callback

    def tag_references(self):
        # Crear una nueva ventana
        self.tag_window = tk.Toplevel(self)
        self.tag_window.title("Etiquetar Referentes")
        self.tag_window.geometry("400x300")
        self.tag_window.configure(bg="#f0f0f0")

        # Crear un cuadro de texto en la nueva ventana
        self.text_box = tk.Text(self.tag_window, wrap=tk.WORD, font=("Arial", 12), bg="#ffffff", fg="#000000")
        self.text_box.pack(expand=True, fill='both', padx=10, pady=10)

        # Botón para guardar el texto ingresado
        self.btn_save = tk.Button(self.tag_window, text="Guardar", command=self.save_text, bg="#4CAF50", fg="white", font=("Arial", 12), relief="raised")
        self.btn_save.pack(pady=10)

    def save_text(self):
        # Obtener el contenido del cuadro de texto
        text_content = self.text_box.get("1.0", tk.END).strip()
        
        if text_content:
            # Hacer algo con el contenido, como guardarlo en un archivo o procesarlo
            messagebox.showinfo("Contenido Guardado", "El texto ha sido guardado.")
            self.tag_window.destroy()
        else:
            messagebox.showwarning("Texto Vacío", "El cuadro de texto está vacío. Por favor, ingrese algún texto.")

    def cancel(self):
        self.on_close_callback()


class ReferenceTaggingWindow(tk.Toplevel):
    def __init__(self, parent):
        super().__init__(parent)
        self.title("Opciones de Etiquetado")
        self.attributes('-zoomed', True)  # Abrir la ventana en pantalla completa
        self.configure(bg="#f0f0f0")

        # Título del menú de etiquetado
        self.label_menu_title = tk.Label(self, text="Menú de Etiquetado", font=("Arial", 24), bg="#f0f0f0")
        self.label_menu_title.pack(pady=20)

        # Botón para abrir las opciones de etiquetado de referentes
        self.btn_tag_options = tk.Button(self, text="Opciones de Etiquetado de Referentes", command=self.open_tagging_options, bg="#FF5733", fg="white", font=("Arial", 16), width=35, height=3, relief="raised")
        self.btn_tag_options.pack(pady=10)

        # Botón para etiquetado morfológico
        self.btn_morphological_tagging = tk.Button(self, text="Etiquetado Morfológico", command=self.morphological_tagging, bg="#4CAF50", fg="white", font=("Arial", 16), width=35, height=3, relief="raised")
        self.btn_morphological_tagging.pack(pady=10)

    def open_tagging_options(self):
        # Abrir la ventana de opciones de etiquetado de referentes
        self.tagging_options_window = tk.Toplevel(self)
        self.tagging_options_window.title("Opciones de Etiquetado de Referentes")
        self.tagging_options_window.attributes('-zoomed', True)
        self.tagging_options_window.configure(bg="#f0f0f0")

        # Crear el frame con las opciones de etiquetado de referentes
        self.tagging_options_frame = ReferenceTaggingOptions(self.tagging_options_window, self.close_tagging_options)
        self.tagging_options_frame.pack(expand=True, fill="both")

    def close_tagging_options(self):
        # Cerrar la ventana de opciones de etiquetado de referentes
        self.tagging_options_window.destroy()

    def morphological_tagging(self):
        messagebox.showinfo("Etiquetado Morfológico", "Funcionalidad de etiquetado morfológico aún no implementada.")


class CEMTaggerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("CEM Tagger")

        # Maximizar la ventana al inicio
        self.root.attributes('-zoomed', True)

        # Crear una etiqueta de bienvenida con el texto adicional
        welcome_title = "Bienvenida al CEM Tagger"
        welcome_text = ("\n"
                        "Este es un prototipo en desarrollo que permite hacer el etiquetado de referentes \n"
                        "y correferentes para un corpus previamente etiquetado morfosintácticamente. \n"
                        "Este proyecto es desarrollado en el Semillero Corpus Ex Machina de la Universidad de Antioquia.")
        self.label_title = tk.Label(root, text=welcome_title, font=("Arial", 24), justify=tk.CENTER)
        self.label_title.pack(pady=10)

        self.label_text = tk.Label(root, text=welcome_text, font=("Arial", 12), justify=tk.LEFT)
        self.label_text.pack(pady=10)

        # Crear los botones más grandes con estilos modificados
        self.convert_button = tk.Button(root, text="Convertir XML a JSON", command=self.convert_xml_to_json, borderwidth=1, relief="raised", width=35, height=3, font=("Arial", 16), bg="#4CAF50", fg="white")
        self.convert_button.place(relx=0.5, rely=0.61, anchor=tk.CENTER)

        self.tag_button = tk.Button(root, text="Etiquetar Referentes y Correferentes", command=self.tag_references, borderwidth=1, relief="raised", width=35, height=3, font=("Arial", 16), bg="#2196F3", fg="white")
        self.tag_button.place(relx=0.5, rely=0.4, anchor=tk.CENTER)

    def convert_xml_to_json(self):
        # Función para convertir XML a JSON (como en tu código anterior)
        pass

    def tag_references(self):
        # Abrir la ventana secundaria para etiquetado de referentes
        reference_tag_window = ReferenceTaggingWindow(self.root)

    def morphological_tagging(self):
        # Función para etiquetado morfológico (como en tu código anterior)
        pass

if __name__ == "__main__":
    root = tk.Tk()
    app = CEMTaggerApp(root)
    root.mainloop()
